In [6]:
import yfinance as yf

symbols = {
    "Nippon Silver BeES": "SILVERBEES.NS",
    "Nippon Gold BeES": "GOLDBEES.NS"
}

def pct_change(prev, current):
    return ((current - prev) / prev) * 100


for name, ticker in symbols.items():

    stock = yf.Ticker(ticker)
    info = stock.info

    # ---- CURRENT PRICE ----
    current_price = info.get("regularMarketPrice")
    # Fallback for current price if regularMarketPrice is not available
    if current_price is None:
        current_price = stock.fast_info.get("lastPrice")
        # As a last resort, try getting the latest close from history
        if current_price is None:
            hist_current = stock.history(period="2d", interval="1d")
            if not hist_current.empty:
                current_price = hist_current.iloc[-1]["Close"]

    # ---- PREVIOUS CLOSE (official NSE close) ----
    prev_close = info.get("previousClose")

    # ---- FALLBACK using candles if API glitches ----
    if prev_close is None:
        # Get last few days to safely cover weekends/holidays for previous close
        hist_prev = stock.history(period="7d", interval="1d")
        if len(hist_prev) >= 2:
            prev_close = hist_prev.iloc[-2]["Close"]

    # Check if we successfully retrieved both prices
    if prev_close is None:
        print(f"\n{name}: Could not determine previous close price.")
        continue
    if current_price is None:
        print(f"\n{name}: Could not determine current price.")
        continue

    change = pct_change(prev_close, current_price)

    print(f"\n{name}")
    print(f"Previous Close : ₹{prev_close:.2f}")
    print(f"Current Price  : ₹{current_price:.2f}")

    if change < 0:
        print(f"📉 Dip         : {change:.2f}%")
    else:
        print(f"📈 Gain        : {change:.2f}%")


Nippon Silver BeES
Previous Close : ₹276.27
Current Price  : ₹298.55
📈 Gain        : 8.06%

Nippon Gold BeES
Previous Close : ₹124.34
Current Price  : ₹127.85
📈 Gain        : 2.82%
